# Start van het project

1. Data uit mongoDB halen om in een csv te steken.

In [17]:
# extract query result to
%matplotlib inline
%reset
low_memory=False
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
import math
import os
import ast

client = MongoClient("mongodb://localhost:27017/")
 
db = client["lifesnaps"]
collection = db["fitbit"]
doc = collection.find_one({"type": "estimated_oxygen_variation"})
print(doc)

{'_id': ObjectId('62cc1f9ab41dcd4b1beae820'), 'id': ObjectId('621e2e8e67b776a24055b564'), 'type': 'estimated_oxygen_variation', 'data': {'timestamp': '05/24/21 01:03:30', 'Infrared to Red Signal Ratio': -4}}


Test query en queries om de data in een list te steken.

In [2]:

# test query
# from bson import ObjectId

# query = {
#     "type": "lightly_active_minutes"
# }
# docs = collection.find(query).limit(10)
# for doc in docs:
#     print(doc)

# Adjust your query as needed



# query = {
#     "type": "sleep"
# }  
# dataSleep = list(collection.find(query))

# query = {
#     "type": "steps"
# }
# dataSteps = list(collection.find(query))

# query = {
#     "type": "lightly_active_minutes"
# }
# dataLightlyActiveMinutes = list(collection.find(query))

# query = {
#     "type": "moderately_active_minutes"
# }
# dataModeratlyActiveMinutes = list(collection.find(query))

# query = {
#     "type": "very_active_minutes"
# }
# dataVeryActiveMinutes = list(collection.find(query))


# Kobe

# query = {
#     "type": "sedentary_minutes"
# }
# dataSedentaryMinutes = list(collection.find(query))

# query = {
#     "type": "Profile"
# }
# dataProfile = list(collection.find(query))

# query = {
#     "type": "Daily Heart Rate Variability Summary"
# }
# dataDailyHeartRateVariabilitySummary = list(collection.find(query)) 

# query = {
#     "type": "resting_heart_rate"
# }
# dataRestingHeartRate = list(collection.find(query))

In [18]:
query = {
    "type": "steps"
}


dataVeryActiveMinutes = list(collection.find(query))

df = pd.DataFrame(dataVeryActiveMinutes)

csv = "steps"

df.to_csv(f"{csv}.csv", index=False)

print(f"Data has been saved to {csv}.csv")

Data has been saved to steps.csv


In [19]:
df_steps = pd.read_csv("steps.csv")

df_steps['data'] = df_steps['data'].apply(eval)
df_steps_expanded = pd.concat([df_steps.drop(columns=['data']), df_steps['data'].apply(pd.Series)], axis=1)
df_steps_expanded = df_steps_expanded.drop(columns=['_id'])

df_steps_expanded.to_csv("steps_expanded.csv", index=False)

De code om de data naar individuele csv om te zetten

In [3]:
# query = {
#     "type": "very_active_minutes"
# }
# dataVeryActiveMinutes = list(collection.find(query))

# df = pd.DataFrame(dataVeryActiveMinutes)

# csv = "very_active_minutes"

# df.to_csv(f"{csv}.csv", index=False)

# print(f"Data has been saved to {csv}.csv")


## Sema database omzetten naar CSV


In [4]:
client = MongoClient("mongodb://localhost:27017/")
 
db = client["lifesnaps"]
collection = db["sema"]
doc = collection.find_one({"data.SURVEY_NAME": "Context and Mood Survey"})
print(doc)

{'_id': ObjectId('62cc7d568e3d174ffc0fbb47'), 'user_id': ObjectId('621e2f1b67b776a240b3d87c'), 'data': {'PARTICIPANT_ID': 's523044002', 'STUDY_ID': 'uk96OixJ-', 'STUDY_NAME': 'RAIS Consortium Experiment v2.0', 'STUDY_VERSION': 5, 'SURVEY_ID': 'BjTcu3Lg-', 'SURVEY_NAME': 'Context and Mood Survey', 'TRIGGER': 'scheduled', 'START_END': 1, 'CREATED_TS': '2021-12-15T15:26:00', 'SCHEDULED_TS': '2021-12-15T15:16:00', 'STARTED_TS': '2021-12-15T15:26:00', 'COMPLETED_TS': '2021-12-15T15:26:00', 'EXPIRED_TS': None, 'UPLOADED_TS': '2021-12-15T15:26:00', 'TOTAL_RT': '10030', 'RAND_PROB': '<no-response>', 'PLACE': 'HOME', 'PLACE_RT': '2440', 'OTHER': '<not-shown>', 'OTHER_RT': '<not-shown>', 'MOOD': 'TENSE/ANXIOUS', 'MOOD_RT': '7590', 'STEPS': '<no-response>', 'STEPS_RT': '<no-response>'}}


In [5]:

# test query
from bson import ObjectId

query = {
    "data.SURVEY_NAME": "Context and Mood Survey"
}
docs = collection.find(query).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('62cc7d568e3d174ffc0fbb47'), 'user_id': ObjectId('621e2f1b67b776a240b3d87c'), 'data': {'PARTICIPANT_ID': 's523044002', 'STUDY_ID': 'uk96OixJ-', 'STUDY_NAME': 'RAIS Consortium Experiment v2.0', 'STUDY_VERSION': 5, 'SURVEY_ID': 'BjTcu3Lg-', 'SURVEY_NAME': 'Context and Mood Survey', 'TRIGGER': 'scheduled', 'START_END': 1, 'CREATED_TS': '2021-12-15T15:26:00', 'SCHEDULED_TS': '2021-12-15T15:16:00', 'STARTED_TS': '2021-12-15T15:26:00', 'COMPLETED_TS': '2021-12-15T15:26:00', 'EXPIRED_TS': None, 'UPLOADED_TS': '2021-12-15T15:26:00', 'TOTAL_RT': '10030', 'RAND_PROB': '<no-response>', 'PLACE': 'HOME', 'PLACE_RT': '2440', 'OTHER': '<not-shown>', 'OTHER_RT': '<not-shown>', 'MOOD': 'TENSE/ANXIOUS', 'MOOD_RT': '7590', 'STEPS': '<no-response>', 'STEPS_RT': '<no-response>'}}
{'_id': ObjectId('62cc7d568e3d174ffc0fbb49'), 'user_id': ObjectId('621e340467b776a2402d7982'), 'data': {'PARTICIPANT_ID': 's926411913', 'STUDY_ID': 'uk96OixJ-', 'STUDY_NAME': 'RAIS Consortium Experiment v2.0', 'ST

In [ ]:
query = {
    "data.SURVEY_NAME": "Context and Mood Survey"
}

data = list(collection.find(query))

df_sleep = pd.DataFrame(data)

csv = "survey_result"

df_sleep.to_csv(f"{csv}.csv", index=False)

print(f"Data has been saved to {csv}.csv")

Data has been saved to survey_result.csv


## Transform data from mood csv into usable format
- make columns from the fields within data

Mood survey word 3 keer per dag afgenomen. 

In [ ]:
csv = "survey_result"

df_sleep = pd.read_csv(f"{csv}.csv")

df_sleep['data'] = df_sleep['data'].apply(eval)
df_expanded = pd.concat(
    [df_sleep.drop(columns=['data']), df_sleep['data'].apply(pd.Series)],
    axis=1
)
df_expanded = df_expanded.drop(columns=['_id'], errors='ignore')

df_expanded.to_csv(f"{csv}_expanded.csv", index=False)


- remove useless columns into new dataframe

“SCHEDULED_TS” (i.e., the timestamp from when the survey was scheduled, e.g., when the participant received the notification) <sup>1</sup>

<sup>1</sup>https://www.nature.com/articles/s41597-022-01764-x#Sec20



In [12]:
df_survey = pd.read_csv('survey_result_expanded.csv')

df_survey_clean = df_survey[[
    'user_id', 'SCHEDULED_TS', 'MOOD'
    ]]

df_survey_clean = df_survey_clean.replace('<no-response>', np.NaN)

df_survey_clean = df_survey_clean.dropna()


df_survey_clean['SCHEDULED_TS'] = pd.to_datetime(df_survey_clean['SCHEDULED_TS']).dt.date

# print(df_survey_clean['MOOD'].unique())

moods = {
        'TENSE/ANXIOUS': -1, 'TIRED': -1, 'RESTED/RELAXED': 1, 'NEUTRAL': 0, 
        'HAPPY': 1, 'ALERT': -1, 'SAD': -1, 'SURPRISE': 0, 'FEAR': -1, 
        'ANGER': -1, 'SADNESS': -1, 'JOY': 1
        }

df_survey_clean['MOOD'] = df_survey_clean['MOOD'].map(moods)

df_survey_clean = df_survey_clean.groupby(['user_id', 'SCHEDULED_TS'], as_index=False)['MOOD'].sum()

df_survey_clean.rename(columns={'user_id': 'id', 'SCHEDULED_TS': 'date', 'MOOD': 'mood'}, inplace=True)

df_survey_clean.head()

# df_survey_clean.to_csv('tst.csv', index=False)

,id,date,mood
0,621e2e8e67b776a24055b564,2021-05-25,0
1,621e2e8e67b776a24055b564,2021-05-26,1
2,621e2e8e67b776a24055b564,2021-05-27,3
3,621e2e8e67b776a24055b564,2021-05-28,1
4,621e2e8e67b776a24055b564,2021-05-29,2


Transformeer alle cleaned dataframes in 1 dataframe


In [ ]:
# very active minutes
df_vam = pd.read_csv('very_active_minutes_expanded.csv')

df_vam = df_vam.drop(columns=['type'])
df_vam['dateTime'] = pd.to_datetime(df_vam['dateTime'], format='%m/%d/%y %H:%M:%S').dt.date
df_vam = df_vam.rename(columns={'dateTime': 'date', 'value': 'very_active_minutes'})


#sleep expanded
df_sleep = pd.read_csv('sleep_expanded.csv')

def extract_stage_minutes(levels_str, stage):
    try:
        levels_str = levels_str.replace('null', 'None')
        levels_dict = ast.literal_eval(levels_str)
        return levels_dict['summary'][stage]['minutes']
    except Exception:
        return None

sleep_minutes = pd.DataFrame({
    'id': df_sleep['id'],
    'date': df_sleep['dateOfSleep'],
    'sleep_deep_minutes': df_sleep['levels'].apply(lambda x: extract_stage_minutes(x, 'deep')),
    'sleep_wake_minutes': df_sleep['levels'].apply(lambda x: extract_stage_minutes(x, 'wake')),
    'sleep_light_minutes': df_sleep['levels'].apply(lambda x: extract_stage_minutes(x, 'light')),
    'sleep_rem_minutes': df_sleep['levels'].apply(lambda x: extract_stage_minutes(x, 'rem')),
})

# steps
df_steps = pd.read_csv('steps_expanded.csv')

df_steps = df_steps.drop(columns=['type'])
df_steps['dateTime'] = pd.to_datetime(df_steps['dateTime']).dt.date
df_steps = df_steps.rename(columns={'dateTime': 'date', 'value': 'total_steps'})
df_steps = df_steps.groupby(['id', 'date'], as_index=False)['total_steps'].sum()

#df resting heart rate
df_resting_heart_rate = pd.read_csv('resting_heart_rate_expanded.csv')

df_resting_heart_rate = df_resting_heart_rate.drop(columns=['type', 'dateTime'])
df_resting_heart_rate = df_resting_heart_rate.rename(columns={'value': 'resting_heart_rate'})


# df daily hr variability summ
df_daily_heart_rate_variability_summary = pd.read_csv('daily_hrv_summary_expanded.csv')

df_daily_heart_rate_variability_summary['dateTime'] = pd.to_datetime(df_daily_heart_rate_variability_summary['timestamp']).dt.date
df_daily_heart_rate_variability_summary = df_daily_heart_rate_variability_summary.drop(columns=['type', 'timestamp'])
df_daily_heart_rate_variability_summary = df_daily_heart_rate_variability_summary.rename(columns={'dateTime': 'date'})

# profile
df_profile = pd.read_csv('profile_expanded.csv')

df_profile = df_profile.drop(columns=['type', 'child', 'time_display_format', 'weight_unit', 'height_unit', 'distance_unit', 'water_unit', 'glucose_unit', 'swim_unit'])

# lightly active minutes
df_lightly_active_minutes = pd.read_csv('lightly_active_minutes_expanded.csv')

df_lightly_active_minutes = df_lightly_active_minutes.drop(columns=['type'])
df_lightly_active_minutes['dateTime'] = pd.to_datetime(df_lightly_active_minutes['dateTime'], format='%m/%d/%y %H:%M:%S').dt.date
df_lightly_active_minutes = df_lightly_active_minutes.rename(columns={'dateTime': 'date', 'value': 'total_lightly_active_minutes'})
df_lightly_active_minutes = df_lightly_active_minutes.groupby(['id', 'date'], as_index=False)['total_lightly_active_minutes'].sum()

#moderatly active minutes
df_moderalty_active_minutes = pd.read_csv('moderately_active_minutes_expanded.csv')

df_moderalty_active_minutes = df_moderalty_active_minutes.drop(columns=['type'])
df_moderalty_active_minutes['dateTime'] = pd.to_datetime(df_moderalty_active_minutes['dateTime'], format='%m/%d/%y %H:%M:%S').dt.date
df_moderalty_active_minutes = df_moderalty_active_minutes.rename(columns={'dateTime': 'date', 'value': 'total_moderately_active_minutes'})
df_moderalty_active_minutes = df_moderalty_active_minutes.groupby(['id', 'date'], as_index=False)['total_moderately_active_minutes'].sum()

#sedentary minutes
df_sedentary_minutes = pd.read_csv('sedentary_minutes_expanded.csv')

df_sedentary_minutes = df_sedentary_minutes.drop(columns=['type'])
df_sedentary_minutes['dateTime'] = pd.to_datetime(df_sedentary_minutes['dateTime'], format='%m/%d/%y %H:%M:%S').dt.date
df_sedentary_minutes = df_sedentary_minutes.rename(columns={'dateTime': 'date', 'value': 'total_sedentary_minutes'})
df_sedentary_minutes = df_sedentary_minutes.groupby(['id', 'date'], as_index=False)['total_sedentary_minutes'].sum()

#